## TP - 04: Vientos

Queremos descargar los datos de vientos de  http://www.remss.com/measurements/wind/ de la misión QuickSCAT. 
El servicio nos ofrence dos servicios de descarga de datos, HTML y FTP. Para bajar una muestra es mas comodo el servicio HTML, pero para descargar mas generales (todo el periodo) nos conviene el servicio de FTP, para ello hay que pedir acceso llenando el siguiente formulario: https://register.remss.com/

Una vez con esto podemos descargar los datos. La pagina tiene un buen manual describiendo que nos podemos encontrar ahí (https://www.remss.com/missions/qscat/), mas allá de lo que vaya resaltando por aca, recomiendo leerlo (o releerlo) en caso de trabajar con los datos de vientos disponibles. Notar que hay scripts para trabajar los datos, asi que vamos a mirar esos scripts primero.

El iniciso nos podia obtener la climatologia mensual, asi que vamos a descargar directamente los datos grillandos con promedios mensuales. 


En la notebook con el código se puede ver como se descargan los scrips y datos. De los scripts, vamos a resaltar 2:  
- quikscat_averaged_v4: Nos permite levantar un archivo (el cual termina siendo una especie de netCDF4).  
- bytemapps: Transforma la imagen en datos. Script principal. 

Uno puede, ademas, descargar scripts con ejemplos de uso, para conectarse ftp, para trabajar con datos diarios y no mensuales, etc.


In [ ]:
import os

import numpy as np

import matplotlib.axes as maxes
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.crs import PlateCarree
from cartopy.feature import LAND, COASTLINE, RIVERS
from cmocean.cm import speed
from mpl_toolkits.axes_grid1 import make_axes_locatable

from winds.quikscat_averaged_v4 import QuikScatAveraged
from winds.ftp import init_ftp, download
from winds.bytemaps import get_uv

def extract_from_region(var, lat, lon, region):
    """
    extract values of a variable betweeen lat,lon coordenates.

    """
    def _order_minmax(point):
        if point[1] > point[0]:
            return point[0], point[1]
        else:
            return point[1], point[0]

    def get_index_limits(vardim, limits):
        return [np.abs(vardim - limit).argmin() for limit in limits]

    def reduce_dim(vardim, idxs):
        m, M =  _order_minmax(idxs)
        return vardim[m:M]

    def extract_var(var, lat_idxs, lon_idxs):
        lat_m, lat_M =  _order_minmax(lat_idxs)
        lon_m, lon_M =  _order_minmax(lon_idxs)
        return var[lat_m : lat_M, lon_m : lon_M]

    lat_idxs = get_index_limits(lat, region["lat"])
    lon_idxs = get_index_limits(lon, region["lon"])
    if type(var) == dict:
        _var = dict()
        for nvar, vvar in var.items():
            _var[nvar] = extract_var(vvar, lat_idxs, lon_idxs)
    else:
        _var = extract_var(var, lat_idxs, lon_idxs)

    return _var, reduce_dim(lat, lat_idxs), reduce_dim(lon, lon_idxs)

def get_winds_data(filename, region=None, missing=-999):
    ds = QuikScatAveraged(filename, missing=missing)
    wspeed = ds.variables["windspd"][:]
    wdir = ds.variables["winddir"][:]
    
    lon = ds.variables['longitude']
    lat = ds.variables['latitude']
    if region:
        wspeed, _, _  = extract_from_region(wspeed, lat, lon, region=region)
        wdir, lat, lon  = extract_from_region(wdir, lat, lon, region=region)
    u, v = get_uv(wspeed, wdir)
    bad = np.where(wspeed < 0)
    u[bad] = 0.
    v[bad] = 0.
    
    return lon, lat, u, v

def get_and_clean_nans(u, v, num=0):
    u[u == -999] = np.nan
    v[v == -999] = np.nan
    N = np.isnan(u).astype(int)
    u = np.nan_to_num(u, num)
    v = np.nan_to_num(v, num)
    return N, u, v

In [ ]:
DESCARGAR_SCRIPTS = False
DESCARGAR_DATOS = False

mycred = {
    "host": "ftp.remss.com",
    "usr": "santi.basanes@gmail.com",
    "pwd": "santi.basanes@gmail.com",
}

ftpscripts = "/qscat/scatterometer_bmap_support/python" 
lsscripts = ['bytemaps.py', 'example_usage.py', 'quikscat_averaged_v4.py', 'quikscat_daily_v4.py', 'readme_qscat.txt']
outscripts = "./winds"

ftpfiles = "/qscat/bmaps_v04"
years = ['y1999',  'y2000', 'y2001', 'y2002', 'y2003', 'y2004', 'y2005',  'y2006',  'y2007', 'y2008', 'y2009']
outfiles =  "./winds_data"

In [ ]:
if (DESCARGAR_DATOS or DESCARGAR_SCRIPTS): 
    wftp = init_ftp(**mycred)

# download scripts in folder ./winds
if  DESCARGAR_SCRIPTS:
    for nf in lsscripts:
        ftppath = os.path.join(ftpscripts, nf)
        localpath = os.path.join(outscripts, nf)
        download(wftp, localpath, ftppath)

# download data in folder winds_data     
if DESCARGAR_DATOS:
    wftp.cwd(ftpfiles)
    for year in years:
        wftp.cwd(year)
        months = wftp.nlst()
        for month in months:
            wftp.cwd(month)
            nfiles = wftp.nlst()
            nf = f"qscat_{year[1:]}{month[1:]}v4.gz"
            if nf in nfiles:
                localpath = os.path.join(outfiles, nf)
                ftppath = os.path.join(ftpfiles, year, month, nf)
                download(wftp, localpath, ftppath)
            wftp.cwd("..")
        wftp.cwd("..")
    wftp.close()

#### Análisis de los vientos

Nos interesa la climatologia mensual de Enero y Julio. Para ello vamos a tomar todos los años y nos vamos a quedar solo con esos meses. Una vez descartado el resto, vamos a promediar los vientos (sus compenetes zonal y meridional) a lo largo del tiempo.  

In [ ]:
MARARG = dict(lat=[-65, -21], lon=[283, 327])

In [ ]:
outfiles =  "./winds_data"
missing = -999

In [ ]:
# PROMEDIOS

years = range(2000, 2010)
month = ["01", "06"] 
for year in years:
    fn1 = os.path.join(outfiles, f"qscat_{year}01v4.gz")
    fn2 = os.path.join(outfiles, f"qscat_{year}06v4.gz")
    if year==2000:
        lon1, lat1, u1, v1 = get_winds_data(fn1, region=MARARG, missing=np.nan)
        lon2, lat2, u2, v2 = get_winds_data(fn2, region=MARARG, missing=np.nan)
        N1, u1, v1 = get_and_clean_nans(u1, v1)
        N2, u2, v2 = get_and_clean_nans(u2, v2)
    else:
        _, _, u, v = get_winds_data(fn1, region=MARARG)
        n, u, v = get_and_clean_nans(u, v)
        N1 += n
        u1 += u
        v1 += v
        _, _,  u, v = get_winds_data(fn2, region=MARARG)
        n,  u, v = get_and_clean_nans(u, v)
        u2 += u
        v2 += v

N1 = 10 - N1
N2 = 10 - N2


u1 =  np.divide(u1, N1, out=np.zeros(N1.shape, dtype=float), where=(N1!=0))
v1 =  np.divide(v1, N1, out=np.zeros(N1.shape, dtype=float), where=(N1!=0))

u2=  np.divide(u2, N2, out=np.zeros(N2.shape, dtype=float), where=(N2!=0))
v2 =  np.divide(v2, N2, out=np.zeros(N2.shape, dtype=float), where=(N2!=0))

wspeed1 =  np.sqrt(u1**2, v1**2)
wspeed2 =  np.sqrt(u2**2, v2**2)

In [ ]:
step = 10
scale = 100
levels = np.arange(0, 20, 0.5)

r1_lonM = -34
r1_lonm = -48
r1_latM = -27
r1_latm = -30 

r2_lonM = -34
r2_lonm = -48
r2_latM = -35
r2_latm = -38


fig, axs = plt.subplots(1, 2, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(10, 10), sharex=True, sharey=True)


axs[0].set_title("Vientos mensual Enero")
im2 = axs[0].contourf(lon1, lat1, wspeed1, vmin=0, vmax=20, levels=levels, cmap=speed, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
qv = axs[0].quiver(lon1[::step], lat1[::step], u1[::step,::step], v1[::step,::step], scale=scale, color='k')


axs[1].set_title("Vientos mensual Junio")
im2 = axs[1].contourf(lon2, lat2, wspeed2, vmin=0, vmax=20, levels=levels, cmap=speed, zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2,  cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
qv = axs[1].quiver(lon2[::step], lat2[::step], u2[::step,::step], v2[::step,::step], scale=scale, color='k')

for i in range(2):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False

axs[1].plot([r1_lonm, r1_lonm, r1_lonM, r1_lonM, r1_lonm], [r1_latM, r1_latm, r1_latm, r1_latM, r1_latM], "r--", label="region 1")
axs[1].plot([r2_lonm, r2_lonm, r2_lonM, r2_lonM, r2_lonm], [r2_latM, r2_latm, r2_latm, r2_latM, r2_latM], "b--",  label="region 2")
axs[0].plot([r1_lonm, r1_lonm, r1_lonM, r1_lonM, r1_lonm], [r1_latM, r1_latm, r1_latm, r1_latM, r1_latM], "r--", label="region 1")
axs[0].plot([r2_lonm, r2_lonm, r2_lonM, r2_lonM, r2_lonm], [r2_latM, r2_latm, r2_latm, r2_latM, r2_latM], "b--",  label="region 2")


axs[0].legend()
axs[1].legend()
plt.show()

print("Figura 1: Climatología vientos. A la izquierda se observa la climatologia del mes de Enero. Mientras a la derecha se encuentra la climatologia mensual de Junio. En ambas figuras, las lineas punteadas \nrepresentan regiones sobre las cuales se hace el gráfico de las rosa de los vientos.")

In [ ]:
from windrose import WindroseAxes

region_1 = dict(
    lat=[-30, -27],
    lon=[312, 326]
)

region_2 = dict(
    lat=[-37, -34],
    lon=[312, 326]
)

In [ ]:
years = range(2000, 2010)
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

lang1 = []
lmag1 = []
lang2 = []
lmag2 = []
for year in [2009]: #years:
    for month in months:
        try:
            fn1 = os.path.join(outfiles, f"qscat_{year}{month}v4.gz")
            fn2 = os.path.join(outfiles, f"qscat_{year}{month}v4.gz")
            _, _, ru1, rv1 = get_winds_data(fn1, region=region_1, missing=np.nan)
            _, _, ru2, rv2 = get_winds_data(fn2, region=region_2, missing=np.nan)
            ang1 = np.arctan2(np.nanmean(ru1), np.nanmean(rv1))*(180/np.pi)
            mag1 = np.sqrt(np.nanmean(ru1)**2 + np.nanmean(rv1)**2)
            lang1.append(ang1)
            lmag1.append(mag1)
            
            ang2 = np.arctan2(np.nanmean(ru2), np.nanmean(rv2))*(180/np.pi)
            mag2 = np.sqrt(np.nanmean(ru2)**2 + np.nanmean(rv2)**2)
            lang2.append(ang2)
            lmag2.append(mag2)
        except KeyError:
            pass
            

lang1 = np.asarray(lang1)
lang2 = np.asarray(lang2)
lmag1 = np.asarray(lmag1)
lmag2 = np.asarray(lmag2)


angs1 = np.remainder(lang1,  360)
angs2  = np.remainder(lang2,  360)

In [ ]:
theta_labels = ["E", "N-E", "N", "N-W", "W", "S-W", "S", "S-E"]
fig, axs = plt.subplots(1, 2, subplot_kw={'projection': "windrose"}, layout='constrained', figsize=(8, 5), sharex=True, sharey=True)
axs[0].bar(angs1, lmag1, normed=True, edgecolor='white')
axs[0].set_legend()
axs[0].set_title('Region 1', fontsize=16)

axs[1].bar(angs2, lmag2, normed=True, edgecolor='white')
axs[1].set_legend()
axs[1].set_title('Region 2', fontsize=16)
plt.show()

print("Figura 2: Rosa de los vientos para la región 1 (Izquierda) y región 2 (derecha). La ubicación de ambas regiones se señala en la figura 1.")